# `AzureContentSafetyTextTool`

In [1]:
from libs.community.langchain_community.tools.azure_cognitive_services.content_safety import AzureContentSafetyTextTool
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain import hub

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

An agent needs a prompt to tell it what to do when given input, this pulls down a base prompt from LangSmith.

In [ ]:
LANGSMITH_KEY = os.environ['LANGSMITH_KEY']
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

The `AzureContentSafetyTextTool` can be passed to an agent with a model and they will be used in conjunction.

In [4]:
tools = [AzureContentSafetyTextTool()]

model = AzureChatOpenAI(
    openai_api_version=os.environ['OPENAI_API_VERSION'],  
    azure_deployment=os.environ['COMPLETIONS_MODEL'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    temperature=0.5,
    max_tokens=200,
    timeout=60,
    max_retries=10,
    )

agent = create_structured_chat_agent(model, tools, prompt)

In [5]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

The tool inherits from LangChain cores `BaseTool` class which defines a class that can be used by a model if it decides it would be useful given a certain input. Below I mention wanting to check text for harmful content and the model decides the tool will be useful here.

In [ ]:
input = "I hate you"
agent_executor.invoke({"input": f"Can you check the following text for harmful content : {input}"})